# Skincare Recommendation Engine


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import heapq


In [ ]:

df = pd.read_csv('/content/newdataset.csv')



In [ ]:
df.head()

,label,url,brand,name,price,skin_type,spf,concern,concern_2,concern_3,key_ingredient,formulation,climate,medication,pregnancy
0,face-moisturisers,https://www.myntra.com/face-moisturisers/lakme...,Lakme,Absolute Perfect Radiance Skin Lightening Day ...,₹ 79,All,NaN,General Care,NaN,NaN,NaN,Cream,Extreme climate(Extremely hot in summers and c...,yes,yes
1,face-moisturisers,https://www.myntra.com/face-moisturisers/bioti...,Biotique,Bio Morning Nectar Flawless Sustainable Skin M...,₹ 165,All,NaN,Uneven Skin Tone,Hydration,Dark Spots,Honey,Lotion,Extreme climate(Extremely hot in summers and c...,yes,yes
2,face-moisturisers,https://www.myntra.com/face-moisturisers/nivea...,Nivea,Unisex Aloe Hydration Skin Cream 100 ml,₹ 92,All,NaN,Dull Skin,Dryness,General Care,Aloe Vera,Cream,Extreme climate(Extremely hot in summers and c...,yes,yes
3,face-moisturisers,https://www.myntra.com/face-moisturisers/vi-jo...,VI-JOHN,Women Set of 5 Saffron Fairness Cream Advanced,₹ 187,All,15 to 30,Softening and Smoothening,Softening and Smoothening,Sun Protection,Vitamin E,Cream,Extreme climate(Extremely hot in summers and c...,yes,yes
4,face-moisturisers,https://www.myntra.com/face-moisturisers/lakme...,Lakme,Peach Milk Soft Creme 150g,₹ 192,All,NaN,General Care,NaN,NaN,NaN,Cream,Extreme climate(Extremely hot in summers and c...,yes,yes


## Data Preprocessing

## Imputation of values

In [ ]:
df['label'].value_counts()

face-moisturisers    300
cleanser             300
mask-and-peel        300
sunscreen            272
eye-cream            100
Name: label, dtype: int64

In [ ]:
df.isna().sum()

label               0
url                 0
brand              42
name               42
price              42
skin_type          43
spf               944
concern            75
concern_2         536
concern_3         719
key_ingredient    610
formulation       378
climate             0
medication          0
pregnancy           0
dtype: int64

In [ ]:
df['concern_2'].fillna('', inplace = True)
df['concern_3'].fillna('', inplace = True)
df['concern'] = df['concern'] + ',' + df['concern_2'] + ',' + df['concern_3']


In [ ]:
df['skin_type'].fillna('', inplace=True)
df['concern'].fillna('', inplace=True)
df['climate'].fillna('', inplace=True)
df['medication'].fillna('', inplace=True)
df['pregnancy'].fillna('', inplace=True)

In [ ]:
df['concern']

0                                          General Care,,
1                   Uneven Skin Tone,Hydration,Dark Spots
2                          Dull Skin,Dryness,General Care
3       Softening and Smoothening,Softening and Smooth...
4                                          General Care,,
                              ...                        
1267                                       Dark Circles,,
1268                Fine Lines and Wrinkles,Dark Circles,
1269                   Dark Circles,Dark Circles,Eye Bags
1270                                                     
1271                        Dark Circles,Eye Bags,Dryness
Name: concern, Length: 1272, dtype: object

In [ ]:

df['concern'].value_counts()

Sun Protection,,                                       199
                                                        75
General Care,,                                          68
Daily Use,,                                             32
Fine Lines and Wrinkles,,                               32
                                                      ... 
Lightening,Dark Spots,                                   1
Daily Use,Acne or Blemishes,Fine Lines and Wrinkles      1
Anti-Pollution,Acne or Blemishes,Excess Oil              1
Excess Oil,Acne or Blemishes,                            1
Lightening,Dryness,                                      1
Name: concern, Length: 432, dtype: int64

In [ ]:
df2 = df[((df['label'] == 'face-moisturisers') | (df['label'] == 'mask-and-peel') | (df['label'] == 'cleanser') | (df['label'] == 'eye-cream'))]
df2
LABELS = list(df2.label.unique())

In [ ]:
df2 = df2[df2['skin_type'].isna() == False]
df2.index = [i for i in range(0, len(df2))]
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   label           1000 non-null   object
 1   url             1000 non-null   object
 2   brand           967 non-null    object
 3   name            967 non-null    object
 4   price           967 non-null    object
 5   skin_type       1000 non-null   object
 6   spf             66 non-null     object
 7   concern         1000 non-null   object
 8   concern_2       1000 non-null   object
 9   concern_3       1000 non-null   object
 10  key_ingredient  564 non-null    object
 11  formulation     666 non-null    object
 12  climate         1000 non-null   object
 13  medication      1000 non-null   object
 14  pregnancy       1000 non-null   object
dtypes: object(15)
memory usage: 125.0+ KB


In [ ]:
df2[df2['concern'].isna() == True]['label'].value_counts()

Series([], Name: label, dtype: int64)

In [ ]:
df2[df2['label'] ==  'eye-cream']['concern'].value_counts()

Fine Lines and Wrinkles,Dark Circles,Eye Bags    16
Dark Circles,Eye Bags,                           12
Dark Circles,,                                   11
Fine Lines and Wrinkles,Dark Circles,             9
Dark Circles,Dark Circles,Eye Bags                8
Dark Circles,Eye Bags,Dryness                     7
Dark Circles,Eye Bags,Dark Circles                6
                                                  4
Fine Lines and Wrinkles,,                         3
Dark Circles,Dryness,                             3
Dryness,Dark Circles,                             3
Fine Lines and Wrinkles,Dryness,                  3
Dryness,,                                         2
Dryness,Eye Bags,Dark Circles                     2
Fine Lines and Wrinkles,Dark Circles,Dryness      2
Lightening,Dark Circles,Eye Bags                  2
Dark Circles,Dark Circles,                        1
Dark Circles,Dryness,Eye Bags                     1
Fine Lines and Wrinkles,Dryness,Eye Bags          1
Lightening,,

In [ ]:
df2['concern'] = df2['concern'].str.lower()
# df2[df2['label'] == 'face-moisturisers']['concern'].value_counts()
# df2[df2['label'] == 'mask-and-peel']['concern'].value_counts()
df2[df2['label'] == 'sunscreen']['concern'].value_counts()



Series([], Name: concern, dtype: int64)

In [ ]:
top_concerns = {'face-moisturisers':'general care', 'mask-and-peel':'daily use', 'cleanser':'general care', 'eye-cream':'fine lines,wrinkles,dark circles,eye bags' }
entries = len(df2)
for i in range(entries):
    label = df2.iloc[i]['label']
    if pd.isnull(df2.iloc[i]['concern']):
        df2.iloc[i]['concern'] = top_concerns[label]
df2.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   label           1000 non-null   object
 1   url             1000 non-null   object
 2   brand           967 non-null    object
 3   name            967 non-null    object
 4   price           967 non-null    object
 5   skin_type       1000 non-null   object
 6   spf             66 non-null     object
 7   concern         1000 non-null   object
 8   concern_2       1000 non-null   object
 9   concern_3       1000 non-null   object
 10  key_ingredient  564 non-null    object
 11  formulation     666 non-null    object
 12  climate         1000 non-null   object
 13  medication      1000 non-null   object
 14  pregnancy       1000 non-null   object
dtypes: object(15)
memory usage: 125.0+ KB


In [ ]:
df2['brand'] = df2['brand'].str.lower()
df2['name'] = df2['name'].str.lower()
df2['skin_type'] = df2['skin_type'].str.lower()
df2['climate'] = df2['climate'].str.lower()
df2['concern'] = df2['concern'].str.replace(' and ', ',').str.replace(' or ', ',')
df2['climate'] = df2['climate'].str.replace(' and ', ',').str.replace(' or ', ',')
df2

,label,url,brand,name,price,skin_type,spf,concern,concern_2,concern_3,key_ingredient,formulation,climate,medication,pregnancy
0,face-moisturisers,https://www.myntra.com/face-moisturisers/lakme...,lakme,absolute perfect radiance skin lightening day ...,₹ 79,all,NaN,"general care,,",,,NaN,Cream,"extreme climate(extremely hot in summers,cold ...",yes,yes
1,face-moisturisers,https://www.myntra.com/face-moisturisers/bioti...,biotique,bio morning nectar flawless sustainable skin m...,₹ 165,all,NaN,"uneven skin tone,hydration,dark spots",Hydration,Dark Spots,Honey,Lotion,"extreme climate(extremely hot in summers,cold ...",yes,yes
2,face-moisturisers,https://www.myntra.com/face-moisturisers/nivea...,nivea,unisex aloe hydration skin cream 100 ml,₹ 92,all,NaN,"dull skin,dryness,general care",Dryness,General Care,Aloe Vera,Cream,"extreme climate(extremely hot in summers,cold ...",yes,yes
3,face-moisturisers,https://www.myntra.com/face-moisturisers/vi-jo...,vi-john,women set of 5 saffron fairness cream advanced,₹ 187,all,15 to 30,"softening,smoothening,softening,smoothening,su...",Softening and Smoothening,Sun Protection,Vitamin E,Cream,"extreme climate(extremely hot in summers,cold ...",yes,yes
4,face-moisturisers,https://www.myntra.com/face-moisturisers/lakme...,lakme,peach milk soft creme 150g,₹ 192,all,NaN,"general care,,",,,NaN,Cream,"extreme climate(extremely hot in summers,cold ...",yes,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,eye-cream,https://www.myntra.com/eye-cream/skinkraft/ski...,skinkraft,vitamin c+e under eye gel for pigmented under ...,₹ 449,all,NaN,"dark circles,,",,,Vitamin E,Gel,"extreme climate(extremely hot in summers,cold ...",yes,yes
996,eye-cream,https://www.myntra.com/eye-cream/casmara/casma...,casmara,eye-cream 15 ml,₹ 2200,all,NaN,"fine lines,wrinkles,dark circles,",Dark Circles,,NaN,Gel,"extreme climate(extremely hot in summers,cold ...",yes,yes
997,eye-cream,https://www.myntra.com/eye-cream/myglamm/mygla...,myglamm,white youthfull hydrating eye cream with water...,₹ 716,all,NaN,"dark circles,dark circles,eye bags",Dark Circles,Eye Bags,Vitamin C,Cream,"extreme climate(extremely hot in summers,cold ...",yes,yes
998,eye-cream,https://www.myntra.com/eye-cream/makeup-revolu...,makeup revolution london,hydro bank hydrating & cooling eye balm,₹ 1050,all,NaN,,,,NaN,Gel,"extreme climate(extremely hot in summers,cold ...",yes,yes


In [ ]:
def concern_elements(comma_sep_concerns):
    words = comma_sep_concerns.split(',')
    for w in words:
        if w != '':
            temp = w.rstrip()
            if temp in concerns:
                concerns[temp] += 1
            else:
                concerns[temp] = 1



# features
list(df2['skin_type'].unique())
concerns = {}
for i in range(entries):
    concern_elements(df2.iloc[i]['concern'])


In [ ]:
def feature_elements(value, feature_dict):
    words = value.split(',')
    for w in words:
        if w != '':
            temp = w.rstrip()
            if temp in feature_dict:
                feature_dict[temp] += 1
            else:
                feature_dict[temp] = 1

# features
skin_types = list(df2['skin_type'].unique())
concerns = {}
climates = {}
medications = {}
pregnancies = {}

for i in range(entries):
    # Extract features for each entry
    skin_type = df2.iloc[i]['skin_type']
    feature_elements(df2.iloc[i]['concern'], concerns)
    feature_elements(df2.iloc[i]['climate'], climates)
    feature_elements(df2.iloc[i]['medication'], medications)
    feature_elements(df2.iloc[i]['pregnancy'], pregnancies)

# Now you can use these dictionaries in your recommendation functions.
# Example usage:
print("Concerns:", concerns)
print("Climates:", climates)
print("Medications:", medications)
print("Pregnancies:", pregnancies)



Concerns: {'general care': 143, 'uneven skin tone': 76, 'hydration': 156, 'dark spots': 71, 'dull skin': 166, 'dryness': 119, 'softening': 112, 'smoothening': 112, 'sun protection': 21, 'irregular textures': 6, 'pigmentation': 71, 'skin inflammation': 11, 'fine lines': 129, 'wrinkles': 129, 'deep nourishment': 106, 'blackheads': 61, 'whiteheads': 61, 'excess oil': 90, 'skin sagging': 20, 'acne': 155, 'blemishes': 155, 'tan removal': 48, 'redness': 15, 'anti-pollution': 60, 'pore care': 110, 'lightening': 27, 'daily use': 149, 'blackheads removal': 4, 'oily skin': 16, 'oil control': 15, 'anti-ageing': 2, 'anti acne scarring': 1, 'skin tightening': 2, 'dark circles': 99, 'eye bags': 56}
Climates: {'extreme climate(extremely hot in summers': 1000, 'cold in winters)': 1000, 'hot all year around': 1000, '\ncold all year around': 1000}
Medications: {'yes': 994, 'no': 6}
Pregnancies: {'yes': 994, 'no': 6}


In [ ]:
def concern_elements(comma_sep_concerns, concern_dict):
    words = comma_sep_concerns.split(',')
    for w in words:
        if w != '':
            temp = w.strip()
            if temp in concern_dict:
                concern_dict[temp] += 1
            else:
                concern_dict[temp] = 1

# Extract features for each entry
for i in range(entries):
    skin_type = df2.iloc[i]['skin_type']
    concern_elements(df2.iloc[i]['concern'], concerns)
    feature_elements(df2.iloc[i]['climate'], climates)
    feature_elements(df2.iloc[i]['medication'], medications)
    feature_elements(df2.iloc[i]['pregnancy'], pregnancies)

# Print sorted concerns
sorted_concerns = sorted(concerns.items(), key=lambda kv: (kv[1], kv[0]), reverse=True)
print("Sorted Concerns:")
for concern, count in sorted_concerns:
    print(f"{concern}: {count}")

# Similarly, you can print sorted climates, medications, and pregnancies if needed
# Print sorted climates
sorted_climates = sorted(climates.items(), key=lambda kv: (kv[1], kv[0]), reverse=True)
print("Sorted Climates:")
for climate, count in sorted_climates:
    print(f"{climate}: {count}")

# Print sorted medications
sorted_medications = sorted(medications.items(), key=lambda kv: (kv[1], kv[0]), reverse=True)
print("Sorted Medications:")
for medication, count in sorted_medications:
    print(f"{medication}: {count}")

# Print sorted pregnancies
sorted_pregnancies = sorted(pregnancies.items(), key=lambda kv: (kv[1], kv[0]), reverse=True)
print("Sorted Pregnancies:")
for pregnancy, count in sorted_pregnancies:
    print(f"{pregnancy}: {count}")


Sorted Concerns:
dull skin: 332
hydration: 312
blemishes: 310
acne: 310
daily use: 298
general care: 286
wrinkles: 258
fine lines: 258
dryness: 238
softening: 224
smoothening: 224
pore care: 220
deep nourishment: 212
dark circles: 198
excess oil: 180
uneven skin tone: 152
pigmentation: 142
dark spots: 142
whiteheads: 122
blackheads: 122
anti-pollution: 120
eye bags: 112
tan removal: 96
lightening: 54
sun protection: 42
skin sagging: 40
oily skin: 32
redness: 30
oil control: 30
skin inflammation: 22
irregular textures: 12
blackheads removal: 8
skin tightening: 4
anti-ageing: 4
anti acne scarring: 2
Sorted Climates:
hot all year around: 2000
extreme climate(extremely hot in summers: 2000
cold in winters): 2000

cold all year around: 2000
Sorted Medications:
yes: 1988
no: 12
Sorted Pregnancies:
yes: 1988
no: 12


In [ ]:
concerns.pop('anti acne scarring')
concerns.pop('anti-ageing')
concerns.pop('skin tightening')
concerns.pop('blackheads removal')
concerns.pop('irregular textures')
concerns.pop('skin inflammation')
concerns.pop('oil control')
concerns.pop('redness')
concerns.pop('oily skin')
concerns.pop('skin sagging')
concerns.pop('sun protection')
concerns.pop('lightening')
concerns.pop('tan removal')
concerns.pop('excess oil')

# concerns.pop('anti acne scarring')
# concerns['scarring'] =
# concerns.remo('blackheads removal')
# concerns.remove('dull skin')
# # concerns.remove('dullness')
# concerns.add('dull')
# concerns.remove('oily skin')
# concerns.remove('excess oil')
# concerns.remove('oil control')
# concerns.add('oil')

180

In [ ]:
import pandas as pd

# Assuming df2 is your DataFrame
df2['concern'].fillna('', inplace=True)
df2['climate'].fillna('', inplace=True)
df2['medication'].fillna('', inplace=True)
df2['pregnancy'].fillna('', inplace=True)

# Extract unique values for skin_type, concern, climate, medication, and pregnancy
skin_types = list(df2['skin_type'].unique())
concerns = {}
climates = {}
medications = {}
pregnancies = {}

# Extract unique values for each parameter
for i in range(len(df2)):
    # Extract features for each entry
    skin_type = df2.iloc[i]['skin_type']
    concern_elements(df2.iloc[i]['concern'], concerns)
    feature_elements(df2.iloc[i]['climate'], climates)
    feature_elements(df2.iloc[i]['medication'], medications)
    feature_elements(df2.iloc[i]['pregnancy'], pregnancies)

# Convert dictionaries to lists
concerns = list(concerns.keys())
climates = list(climates.keys())
medications = list(medications.keys())
pregnancies = list(pregnancies.keys())

# Print unique values for each parameter
print("Unique Skin Types:", skin_types)
print("Unique Concerns:", concerns)
print("Unique Climates:", climates)
print("Unique Medications:", medications)
print("Unique Pregnancies:", pregnancies)

# Continue with the existing logic for one-hot encoding and recommendation

# Rest of your code...


Unique Skin Types: ['all', 'normal', 'dry', 'oily', '', 'combination', 'sensitive']
Unique Concerns: ['general care', 'uneven skin tone', 'hydration', 'dark spots', 'dull skin', 'dryness', 'softening', 'smoothening', 'sun protection', 'irregular textures', 'pigmentation', 'skin inflammation', 'fine lines', 'wrinkles', 'deep nourishment', 'blackheads', 'whiteheads', 'excess oil', 'skin sagging', 'acne', 'blemishes', 'tan removal', 'redness', 'anti-pollution', 'pore care', 'lightening', 'daily use', 'blackheads removal', 'oily skin', 'oil control', 'anti-ageing', 'anti acne scarring', 'skin tightening', 'dark circles', 'eye bags']
Unique Climates: ['extreme climate(extremely hot in summers', 'cold in winters)', 'hot all year around', '\ncold all year around']
Unique Medications: ['yes', 'no']
Unique Pregnancies: ['yes', 'no']


In [ ]:
def concern_elements(comma_sep_concerns, concern_dict):
    words = comma_sep_concerns.split(',')
    for w in words:
        if w != '':
            temp = w.rstrip()
            if temp in concern_dict:
                concern_dict[temp] += 1
            else:
                concern_dict[temp] = 1


In [ ]:
features = list(df2['skin_type'].unique()) + list(concerns) + ['climate', 'medication', 'pregnancy']
(features)

features = ['normal', 'dry', 'oily', 'combination', 'acne', 'sensitive', 'fine lines', 'wrinkles', 'redness', 'dull', 'pore', 'pigmentation', 'blackheads', 'whiteheads', 'blemishes', 'dark circles', 'eye bags', 'dark spots', 'climate_Extreme', 'climate_Hot', 'climate_Cold', 'medication_yes', 'medication_no', 'pregnancy_yes', 'pregnancy_no']


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   label           1000 non-null   object
 1   url             1000 non-null   object
 2   brand           967 non-null    object
 3   name            967 non-null    object
 4   price           967 non-null    object
 5   skin_type       1000 non-null   object
 6   spf             66 non-null     object
 7   concern         1000 non-null   object
 8   concern_2       1000 non-null   object
 9   concern_3       1000 non-null   object
 10  key_ingredient  564 non-null    object
 11  formulation     666 non-null    object
 12  climate         1000 non-null   object
 13  medication      1000 non-null   object
 14  pregnancy       1000 non-null   object
dtypes: object(15)
memory usage: 125.0+ KB


In [ ]:
len(features)

25

In [ ]:
def search_concern(target, i):
    if target in df2.iloc[i]['concern']:
        return True
    return False

one_hot_encodings = np.zeros([entries, len(features)])

# Skin types first
for i in range(entries):
    for j in range(5):
        target = features[j]
        sk_type = df2.iloc[i]['skin_type']
        if sk_type == 'all':
            one_hot_encodings[i][0:5] = 1
        elif target == sk_type:
            one_hot_encodings[i][j] = 1

# Other features
for i in range(entries):
    for j in range(5, len(features)):
        feature = features[j]
        if feature in df2.iloc[i]['concern']:
            one_hot_encodings[i][j] = 1

        # Adding values for climate, medication, and pregnancy
        if feature == 'climate_Extreme':
            one_hot_encodings[i][j] = 1 if 'Extreme' in df2.iloc[i]['climate'] else 0
        elif feature == 'climate_Hot':
            one_hot_encodings[i][j] = 1 if 'Hot' in df2.iloc[i]['climate'] else 0
        elif feature == 'climate_Cold':
            one_hot_encodings[i][j] = 1 if 'Cold' in df2.iloc[i]['climate'] else 0
        elif feature == 'medication_yes':
            one_hot_encodings[i][j] = 1 if df2.iloc[i]['medication'] == 'yes' else 0
        elif feature == 'medication_no':
            one_hot_encodings[i][j] = 1 if df2.iloc[i]['medication'] == 'no' else 0
        elif feature == 'pregnancy_yes':
            one_hot_encodings[i][j] = 1 if df2.iloc[i]['pregnancy'] == 'yes' else 0
        elif feature == 'pregnancy_no':
            one_hot_encodings[i][j] = 1 if df2.iloc[i]['pregnancy'] == 'no' else 0


In [ ]:
x = one_hot_encodings[456]

# Extract values for climate
x[features.index('climate_Extreme')] = 1 if 'Extreme' in df2.iloc[456]['climate'] else 0
x[features.index('climate_Hot')] = 1 if 'Hot' in df2.iloc[456]['climate'] else 0
x[features.index('climate_Cold')] = 1 if 'Cold' in df2.iloc[456]['climate'] else 0

# Extract values for medication and pregnancy
x[features.index('medication_yes')] = 1 if df2.iloc[456]['medication'] == 'yes' else 0
x[features.index('medication_no')] = 1 if df2.iloc[456]['medication'] == 'no' else 0
x[features.index('pregnancy_yes')] = 1 if df2.iloc[456]['pregnancy'] == 'yes' else 0
x[features.index('pregnancy_no')] = 1 if df2.iloc[456]['pregnancy'] == 'no' else 0


In [ ]:
x = one_hot_encodings[456]

In [ ]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(one_hot_encodings)
distances, indices = nbrs.kneighbors(one_hot_encodings)

# get_index_from_name('peach milk soft creme 150g')


In [ ]:
# finding the closes data points
def recs_nn(query=None,id=None):
    if id:
        for id in indices[id][1:]:
            print(anime.ix[id]["name"])
    if query:
        found_id = get_index_from_name(query)
        for id in indices[found_id][1:]:
            print(index2prod(id))

## Cosine Similarity

In [ ]:
# utility functions
def name2index(name):
    return df2[df2["name"]==name].index.tolist()[0]

def index2prod(index):
    return df2.iloc[index]

In [ ]:
def wrap(info_arr):
    result = {}
    result['brand'] = info_arr[0]
    result['name'] = info_arr[1]
    result['price'] = info_arr[2]
    result['url'] = info_arr[3]
    result['skin_type'] = info_arr[4]
    result['concern'] = str(info_arr[5]).split(',')
    result['climate'] = info_arr[6]  # Add climate parameter
    result['medication'] = info_arr[7]  # Add medication parameter
    result['pregnancy'] = info_arr[8]  # Add pregnancy parameter
    return result

# Recommend top similar items from a category
def recs_cs(vector=None, name=None, label=None, count=5):
    products = []
    if name:
        idx = name2index(name)
        fv = one_hot_encodings[idx]
    elif vector:
        fv = vector
    cs_values = cosine_similarity(np.array([fv, ]), one_hot_encodings)
    df2['cs'] = cs_values[0]

    if label:
        dff = df2[df2['label'] == label]
    else:
        dff = df2

    if name:
        dff = dff[dff['name'] != name]
    recommendations = dff.sort_values('cs', ascending=False).head(count)

    data = recommendations[['brand', 'name', 'price', 'url', 'skin_type', 'concern', 'climate', 'medication', 'pregnancy']].to_dict('split')['data']
    for element in data:
        products.append(wrap(element))
    return products


In [ ]:
# Overall recommendation
def recs_essentials(vector=None, name=None):
    response = {}
    for label in LABELS:
        if name:
            r = recs_cs(None, name, label)
        elif vector:
            r = recs_cs(vector, None, label)
        response[label] = r
    return response


In [ ]:
# overall recommendation
def recs_essentials(vector = None, name = None):
#     print("ESSENTIALS:")
    response = {}
    for label in LABELS:
#         print(f"{label}:")
        if name:
            r = recs_cs(None, name, label)
        elif vector:
            r = recs_cs(vector, None, label)
        response[label] = r
    return response


In [ ]:
# Updated features list including the new parameters
features = ['normal', 'dry', 'oily', 'combination', 'acne', 'sensitive', 'fine lines', 'wrinkles', 'redness', 'dull', 'pore',
            'pigmentation', 'blackheads', 'whiteheads', 'blemishes', 'dark circles', 'eye bags', 'dark spots', 'climate',
            'medication', 'pregnancy']

# Sample vector with the new parameters
x = [1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1]  # Update values for climate, medication, pregnancy

# Get recommendations
y = recs_essentials(x, None)
y

{'face-moisturisers': [{'brand': 'azani active care',
   'name': 'unisex acne rescue cream - 30 ml',
   'price': '₹ 399',
   'url': 'https://www.myntra.com/face-moisturisers/azani-active-care/azani-active-care-unisex-acne-rescue-cream---30-ml/15322518/buy',
   'skin_type': 'all',
   'concern': ['deep nourishment', 'acne', 'blemishes', 'dull skin'],
   'climate': 'extreme climate(extremely hot in summers,cold in winters),hot all year around,\ncold all year around',
   'medication': 'yes',
   'pregnancy': 'yes'},
  {'brand': 'mamaearth',
   'name': 'vitamin c face milk with peach for skin illumination 100 ml',
   'price': '₹ 404',
   'url': 'https://www.myntra.com/face-moisturisers/mamaearth/mamaearth-vitamin-c-face-milk-with-peach-for-skin-illumination-100-ml/12411986/buy',
   'skin_type': 'all',
   'concern': ['acne', 'blemishes', 'pigmentation', 'dull skin'],
   'climate': 'extreme climate(extremely hot in summers,cold in winters),hot all year around,\ncold all year around',
   'medic

In [ ]:
pip install scikit-learn

In [ ]:
pip install scikit-surprise


In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib inline

import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
# CountVectorizer+tfidfTransformer = tfidfVectorizer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn import metrics
from sklearn.model_selection import train_test_split
from matplotlib import cm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity

from bokeh.io import show,curdoc,output_notebook,push_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource,HoverTool,Select,Paragraph,TextInput
from ipywidgets import interact


import string
from os import path
from PIL import Image
from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator
from sklearn.pipeline import Pipeline

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

import warnings; warnings.simplefilter('ignore')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
y=df.skin_type
x=df.concern

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.25,random_state=42)

In [ ]:
skin_type=['All','Normal','Dry','Combination','Oily']

In [ ]:
nb=Pipeline([('vect',CountVectorizer()),('tfidf',TfidfTransformer()),('clf',MultinomialNB())])

In [ ]:
nb.fit(xtrain,ytrain)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming df is your DataFrame
# Replace NaN values in relevant columns
df['skin_type'].fillna('', inplace=True)
df['concern'].fillna('', inplace=True)

# Convert categorical variables to numerical representations
df['skin_type'] = pd.Categorical(df['skin_type']).codes
df['concern'] = pd.Categorical(df['concern']).codes

# Prepare the features (X) and labels (y)
X = df[['skin_type', 'concern']]
y = df['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a k-Nearest Neighbors classifier (KNN)
knn = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors (n_neighbors) as needed
knn.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = knn.predict(X_test)
y
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Additional metrics
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 70.59%
Classification Report:
                   precision    recall  f1-score   support

         cleanser       0.68      0.69      0.68        70
        eye-cream       0.61      0.90      0.73        21
face-moisturisers       0.68      0.55      0.61        65
    mask-and-peel       0.66      0.66      0.66        53
        sunscreen       0.89      0.91      0.90        46

         accuracy                           0.71       255
        macro avg       0.70      0.74      0.72       255
     weighted avg       0.71      0.71      0.70       255



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming df is your DataFrame
# Replace NaN values in relevant columns
df['skin_type'].fillna('', inplace=True)
df['concern'].fillna('', inplace=True)

# Convert categorical variables to numerical representations
df['skin_type'] = pd.Categorical(df['skin_type']).codes
df['concern'] = pd.Categorical(df['concern']).codes

# Prepare the features (X) and labels (y)
X = df[['skin_type', 'concern']]
y = df['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Decision Tree classifier
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = decision_tree.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Additional metrics
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 72.55%
Classification Report:
                   precision    recall  f1-score   support

         cleanser       0.66      0.69      0.67        70
        eye-cream       0.72      0.86      0.78        21
face-moisturisers       0.64      0.60      0.62        65
    mask-and-peel       0.75      0.72      0.73        53
        sunscreen       0.93      0.91      0.92        46

         accuracy                           0.73       255
        macro avg       0.74      0.75      0.75       255
     weighted avg       0.73      0.73      0.72       255



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming df is your DataFrame
# Replace NaN values in relevant columns
df['climate'].fillna('', inplace=True)
df['concern'].fillna('', inplace=True)

# Convert categorical variables to numerical representations
df['climate'] = pd.Categorical(df['climate']).codes
df['concern'] = pd.Categorical(df['concern']).codes

# Prepare the features (X) and labels (y)
X = df[['climate', 'concern']]
y = df['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest classifier
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust the number of estimators (n_estimators) as needed
random_forest.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = random_forest.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Additional metrics
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 74.12%
Classification Report:
                   precision    recall  f1-score   support

         cleanser       0.68      0.70      0.69        70
        eye-cream       0.86      0.86      0.86        21
face-moisturisers       0.64      0.65      0.64        65
    mask-and-peel       0.76      0.72      0.74        53
        sunscreen       0.91      0.91      0.91        46

         accuracy                           0.74       255
        macro avg       0.77      0.77      0.77       255
     weighted avg       0.74      0.74      0.74       255



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report



# Convert categorical variables to numerical representations
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])
df['skin_type'] = pd.Categorical(df['skin_type']).codes
df['concern'] = pd.Categorical(df['concern']).codes
df['climate'] = pd.Categorical(df['climate']).codes
df['medication'] = pd.Categorical(df['medication']).codes
df['pregnancy'] = pd.Categorical(df['pregnancy']).codes

# Prepare the features (X) and labels (y)
X = df[['skin_type', 'concern', 'climate', 'medication', 'pregnancy']]
y = df['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a K-Nearest Neighbors classifier
knn = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors (k) as needed
knn.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = knn.predict(X_test)

# Convert numerical predictions back to original labels
y_pred_labels = label_encoder.inverse_transform(y_pred)

# Convert y_test to numeric labels
y_test_numeric = label_encoder.transform(y_test)

# Evaluate the model
accuracy = accuracy_score(y_test_numeric, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Additional metrics
print("Classification Report:")
print(classification_report(y_test_numeric, y_pred))


ValueError: y contains previously unseen labels: 2

In [ ]:
from sklearn.neighbors import NearestNeighbors

# Assuming you have 'one_hot_encodings' and 'df2' as your data
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(one_hot_encodings)
distances, indices = nbrs.kneighbors(one_hot_encodings)

# Utility functions
def get_index_from_name(product_name):
    return df2[df2['name'] == product_name].index.tolist()[0]

def index2product(index):
    return df2.iloc[index]

def wrap(info_arr):
    result = {}
    result['brand'] = info_arr[0]
    result['name'] = info_arr[1]
    result['price'] = info_arr[2]
    result['url'] = info_arr[3]
    result['skin_type'] = info_arr[4]
    result['concern'] = str(info_arr[5]).split(',')
    return result

# Recommendation function
def recs_nn(query=None, product_id=None):
    products = []
    if product_id:
        for id in indices[product_id][1:]:
            products.append(wrap(df2.iloc[id]))
    if query:
        found_id = get_index_from_name(query)
        for id in indices[found_id][1:]:
            products.append(wrap(df2.iloc[id]))
    return products

# Example usage
product_recommendations = recs_nn(query='peach milk soft creme 150g')
print(product_recommendations)


[{'brand': 'face-moisturisers', 'name': 'https://www.myntra.com/face-moisturisers/neutriderm/neutriderm-vitamin-e-moisturising-lotion-125ml/13176342/buy', 'price': 'neutriderm', 'url': 'vitamin e moisturising lotion 125ml', 'skin_type': '₹ 850', 'concern': ['all']}, {'brand': 'face-moisturisers', 'name': 'https://www.myntra.com/face-moisturisers/good-vibes/good-vibes-revitalising-face-cream---green-tea-50-g/15283688/buy', 'price': 'good vibes', 'url': 'revitalising face cream - green tea (50 g)', 'skin_type': '₹ 212', 'concern': ['all']}, {'brand': 'face-moisturisers', 'name': 'https://www.myntra.com/face-moisturisers/vi-john/vi-john-women-set-of-5-saffron-fairness-cream-advanced/15390390/buy', 'price': 'vi-john', 'url': 'women set of 5 saffron fairness cream advanced', 'skin_type': '₹ 187', 'concern': ['all']}, {'brand': 'face-moisturisers', 'name': 'https://www.myntra.com/face-moisturisers/nivea/nivea-soft-light-moisturizer-with-vitamin-e-100ml/2273976/buy', 'price': 'nivea', 'url': 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming df is your DataFrame
# Replace NaN values in relevant columns
df['skin_type'].fillna('', inplace=True)
df['concern'].fillna('', inplace=True)

# Convert categorical variables to numerical representations
df['skin_type'] = pd.Categorical(df['skin_type']).codes
df['concern'] = pd.Categorical(df['concern']).codes

# Prepare the features (X) and labels (y)
X = df[['skin_type', 'concern', 'label']]
y = df['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a K-Nearest Neighbors classifier
knn = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors (k) as needed
knn.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = knn.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Additional metrics
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 85.10%
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.84      0.88        70
           1       0.76      0.90      0.83        21
           2       0.77      0.83      0.80        65
           3       0.84      0.77      0.80        53
           4       0.94      0.96      0.95        46

    accuracy                           0.85       255
   macro avg       0.85      0.86      0.85       255
weighted avg       0.86      0.85      0.85       255



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming df is your DataFrame
# Replace NaN values in relevant columns
df['skin_type'].fillna('', inplace=True)
df['concern'].fillna('', inplace=True)
df['climate'].fillna('', inplace=True)
df['medication'].fillna('', inplace=True)
df['pregnancy'].fillna('', inplace=True)

# Convert categorical variables to numerical representations
df['skin_type'] = pd.Categorical(df['skin_type']).codes
df['concern'] = pd.Categorical(df['concern']).codes
df['climate'] = pd.Categorical(df['climate']).codes
df['medication'] = pd.Categorical(df['medication']).codes
df['pregnancy'] = pd.Categorical(df['pregnancy']).codes

# Prepare the features (X) and labels (y)
X = df[['skin_type', 'concern', 'climate', 'medication', 'pregnancy','label']]
y = df['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a K-Nearest Neighbors classifier
knn = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors (k) as needed
knn.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = knn.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Additional metrics
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 85.10%
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.84      0.88        70
           1       0.76      0.90      0.83        21
           2       0.77      0.83      0.80        65
           3       0.84      0.77      0.80        53
           4       0.94      0.96      0.95        46

    accuracy                           0.85       255
   macro avg       0.85      0.86      0.85       255
weighted avg       0.86      0.85      0.85       255



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Assuming df is your DataFrame
# Replace NaN values in relevant columns
df['skin_type'].fillna('', inplace=True)
df['concern'].fillna('', inplace=True)
df['climate'].fillna('', inplace=True)
df['medication'].fillna('', inplace=True)
df['pregnancy'].fillna('', inplace=True)

# Convert categorical variables to numerical representations
df['skin_type'] = pd.Categorical(df['skin_type']).codes
df['concern'] = pd.Categorical(df['concern']).codes
df['climate'] = pd.Categorical(df['climate']).codes
df['medication'] = pd.Categorical(df['medication']).codes
df['pregnancy'] = pd.Categorical(df['pregnancy']).codes
df['label'] = pd.Categorical(df['label']).codes

# Prepare the features (X) and labels (y)
X = df[['skin_type', 'concern', 'climate', 'medication', 'pregnancy','label']]
y = df['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Naive Bayes classifier (MultinomialNB)
nb = MultinomialNB()
nb.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = nb.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Additional metrics
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 63.53%
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        70
           1       0.00      0.00      0.00        21
           2       0.53      0.32      0.40        65
           3       0.55      0.64      0.59        53
           4       0.45      0.80      0.57        46

    accuracy                           0.64       255
   macro avg       0.50      0.55      0.51       255
weighted avg       0.60      0.64      0.60       255



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Assuming df is your DataFrame
# Replace NaN values in relevant columns
df['skin_type'].fillna('', inplace=True)
df['concern'].fillna('', inplace=True)
df['climate'].fillna('', inplace=True)
df['medication'].fillna('', inplace=True)
df['pregnancy'].fillna('', inplace=True)

df['formulation'].fillna('', inplace=True)
df['brand'].fillna('', inplace=True)

# Convert categorical variables to numerical representations
df['skin_type'] = pd.Categorical(df['skin_type']).codes
df['concern'] = pd.Categorical(df['concern']).codes
df['climate'] = pd.Categorical(df['climate']).codes
df['medication'] = pd.Categorical(df['medication']).codes
df['pregnancy'] = pd.Categorical(df['pregnancy']).codes

df['formulation'] = pd.Categorical(df['formulation']).codes
df['brand'] = pd.Categorical(df['brand']).codes
df['label'] = pd.Categorical(df['label']).codes

# Prepare the features (X) and labels (y)
X = df[['skin_type', 'concern', 'climate', 'medication', 'pregnancy', 'formulation', 'brand','label']]
y = df['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Naive Bayes classifier (MultinomialNB)
nb = MultinomialNB()
nb.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = nb.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Additional metrics
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 44.31%
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.36      0.53        70
           1       0.21      0.71      0.33        21
           2       0.58      0.28      0.38        65
           3       0.79      0.49      0.60        53
           4       0.31      0.63      0.41        46

    accuracy                           0.44       255
   macro avg       0.58      0.49      0.45       255
weighted avg       0.66      0.44      0.47       255



In [ ]:
# Print the column names
print(df.columns)


Index(['label', 'url', 'brand', 'name', 'price', 'skin_type', 'spf', 'concern',
       'concern_2', 'concern_3', 'key_ingredient', 'formulation', 'climate',
       'medication', 'pregnancy'],
      dtype='object')


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming df is your DataFrame
# Replace NaN values in relevant columns
df['skin_type'].fillna('', inplace=True)
df['concern'].fillna('', inplace=True)
df['climate'].fillna('', inplace=True)
df['medication'].fillna('', inplace=True)
df['pregnancy'].fillna('', inplace=True)

df['formulation'].fillna('', inplace=True)
df['brand'].fillna('', inplace=True)
df['key_ingredient'].fillna('', inplace=True)  # Add this line to handle NaN values in key_ingredient

# Convert categorical variables to numerical representations
df['skin_type'] = pd.Categorical(df['skin_type']).codes
df['concern'] = pd.Categorical(df['concern']).codes
df['climate'] = pd.Categorical(df['climate']).codes
df['medication'] = pd.Categorical(df['medication']).codes
df['pregnancy'] = pd.Categorical(df['pregnancy']).codes

df['formulation'] = pd.Categorical(df['formulation']).codes
df['brand'] = pd.Categorical(df['brand']).codes
df['key_ingredient'] = pd.Categorical(df['key_ingredient']).codes  # Add this line to convert key_ingredient

df['label'] = pd.Categorical(df['label']).codes

# Prepare the features (X) and labels (y)
X = df[['skin_type', 'concern', 'climate', 'medication', 'pregnancy', 'formulation', 'brand', 'key_ingredient','label']]
y = df['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a K-Nearest Neighbors classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = knn.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Additional metrics
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 68.63%
Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.57      0.63        70
           1       0.70      0.76      0.73        21
           2       0.64      0.68      0.66        65
           3       0.56      0.60      0.58        53
           4       0.86      0.93      0.90        46

    accuracy                           0.69       255
   macro avg       0.69      0.71      0.70       255
weighted avg       0.69      0.69      0.68       255

